<a href="https://colab.research.google.com/github/pumpzcc/EvolutionaryDeepLearning/blob/main/EDL_11_1_FrozenLake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gym
!pip install numpy==1.24.4

In [2]:
#@title Imports
import numpy as np
import gym
import random

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [3]:
#@title Create the Environment
env = gym.make("FrozenLake-v1")

/usr/local/lib/python3.11/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.11/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [4]:
#title Get Action/State sizes
action_size = env.action_space.n
state_size = env.observation_space.n
print(action_size, state_size)

4 16


In [5]:
# Build Q table
Q = np.zeros((state_size, action_size))
print(Q)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [6]:
#@title Hyperparameters
total_episodes = 20000       # Total episodes
learning_rate = 0.7          # Learning rate
max_steps = 99               # Max steps per episode
gamma = 0.95                 # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability
decay_rate = 0.005            # Exponential decay rate for exploration prob

In [7]:
def choose_action(state):
  if random.uniform(0, 1) > epsilon:
    return np.argmax(Q[state,:])
  else:
    return env.action_space.sample()

In [8]:
#@title Learn
def learn(reward, state, action, new_state):
  # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
  # qtable[new_state,:] : all the actions we can take from new state
  Q[state, action] = Q[state, action] + learning_rate * (reward + gamma * np.max(Q[new_state, :]) - Q[state, action])

In [9]:
#@title Train the Agent
# List of rewards
rewards = []
epsilon = 1.0

# 2 For life or until learning is stopped
for episode in range(total_episodes):
    # Reset the environment
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0

    for step in range(max_steps):
        action = choose_action(state)

        #excute the action
        new_state, reward, done, info = env.step(action)

        learn(reward, state, action, new_state)

        total_rewards += reward
        # Our new state is state
        state = new_state

        # If done (if we're dead) : finish episode
        if done == True:
            break

    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
    rewards.append(total_rewards)


print ("Score over time: " +  str(sum(rewards)/total_episodes))
print(Q)

/usr/local/lib/python3.11/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Score over time: 0.4928
[[3.01367967e-02 6.53416735e-02 9.36226492e-02 2.89326849e-02]
 [9.55344428e-03 7.83552307e-04 1.51824904e-02 1.17294967e-01]
 [1.43254427e-02 1.59851313e-01 9.37110723e-03 1.44459970e-02]
 [7.44745479e-03 3.90923047e-03 7.57626097e-03 6.00489726e-02]
 [1.44757053e-01 1.77900683e-02 1.79761829e-02 3.23886886e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [2.59810107e-03 2.97955293e-08 1.61575598e-01 7.89589922e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.45953811e-02 2.14893936e-02 2.80459953e-02 2.87142466e-01]
 [4.25454773e-02 4.99767592e-01 4.00445511e-02 3.85566460e-02]
 [6.83349322e-01 2.82504773e-02 2.33629914e-02 8.35833467e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [6.42784386e-02 7.13794908e-02 4.82087513e-01 5.94420894e-02]
 [1.47760991e-01 8.52070090e-01 2.94005861e-01 1.90074051e-01]
 [0.00000000e+00 0.00000000e+00

In [10]:
#@title Run the agent
env.reset()

for episode in range(5):
    state = env.reset()
    steps = 0
    done = False
    print("****************************************************")
    print("EPISODE ", episode)

    for step in range(max_steps):

        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(Q[state,:])

        new_state, reward, done, info = env.step(action)

        if done:
            # Here, we decide to only print the last state (to see if our agent is on the goal or fall into an hole)
            env.render()
            if new_state == 15:
                print("Goal reached 🏆")
            else:
                print("Aaaah ☠️")

            # We print the number of step it took.
            print("Number of steps", step)

            break
        state = new_state
env.close()

****************************************************
EPISODE  0


/usr/local/lib/python3.11/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.11/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.11/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_reso

Aaaah ☠️
Number of steps 35
****************************************************
EPISODE  1


/usr/local/lib/python3.11/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


Goal reached 🏆
Number of steps 66
****************************************************
EPISODE  2
Aaaah ☠️
Number of steps 75
****************************************************
EPISODE  3
Goal reached 🏆
Number of steps 52
****************************************************
EPISODE  4
Aaaah ☠️
Number of steps 75


In [11]:
#@title Train the Agent
# List of rewards
rewards = []
epsilon = 1.0

# 2 For life or until learning is stopped
for episode in range(total_episodes):
    # Reset the environment
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0

    for step in range(max_steps):
        action = choose_action(state)

        #excute the action
        new_state, reward, done, info = env.step(action)

        learn(reward, state, action, new_state)

        total_rewards += reward
        # Our new state is state
        state = new_state

        # If done (if we're dead) : finish episode
        if done == True:
            break

    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
    rewards.append(total_rewards)


print ("Score over time: " +  str(sum(rewards)/total_episodes))
print(Q)

Score over time: 0.5037
[[3.60657787e-01 1.78623121e-01 8.34435773e-02 7.77370263e-02]
 [2.54983772e-02 1.72216349e-02 1.71839288e-02 3.74702719e-01]
 [1.73490609e-01 1.72147241e-02 2.75829399e-02 3.71172316e-02]
 [1.79481917e-02 4.92663062e-03 1.12575700e-03 3.72690130e-02]
 [3.11048580e-01 4.53929949e-02 1.40467394e-01 1.84497011e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.68014816e-03 2.02532318e-02 9.85142453e-02 6.07518952e-06]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [4.27169646e-02 7.72590628e-02 7.01287460e-02 2.57644681e-01]
 [1.31892357e-01 5.35155840e-01 2.01580878e-02 2.45921711e-02]
 [5.30219738e-01 2.01268745e-02 8.70818365e-04 1.12264362e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.04274696e-01 1.66745212e-01 7.80335959e-01 1.72410801e-01]
 [2.00524520e-01 9.73909635e-01 6.20667401e-01 2.32829004e-01]
 [0.00000000e+00 0.00000000e+00